In [2]:
import bb
from threading import Thread
import queue
import system_model
import numpy as np
import bbrbdl
import control
import linear_controller
from bb_interactive import *

BB = begin()
M = bbrbdl.BBModel()

Masses:  0.6361725123519333 1


/home/soa/bb/bbrbdl.py:25: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  body = rbdl.Body.fromMassComInertia(roller_mass, np.array([0.,0.,0.]), np.diag([1, 1, roller_moi]))
/home/soa/bb/bbrbdl.py:36: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  body = rbdl.Body.fromMassComInertia(board_mass, np.array([0, 0.5, 0]), np.eye(3)*.1)
/home/soa/bb/bbrbdl.py:45: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  body = rbdl.Body.fromMassComInertia(body_mass, np.array([0., 0, 0]), np.eye(3) * body_moi)


Starting GL
GL Started
Masses:  0.6361725123519333 1


In [2]:
ns = 6
s = system_model.Simulation(M.model,np.zeros(ns))
X = s.sim(5)

lsa = system_model.LinearSystemApproximator()
ss = lsa.calculate(M.model, s.x0[:3], s.x0[3:], np.zeros(ns/2))

css = control.ss(ss.A, ss.B, np.eye(ns), np.zeros([ns,ns/2]))

[K,S,E] = control.lqr(css, np.diag([10, 10, 10, 1, 1, 1]), np.eye(ns/2))
P = linear_controller.LinearController( K, np.zeros(ns), np.zeros(ns/2))
s.control = P
s.x0[0] = -.05
s.reset()
X = s.simUntil()

BB.show_state(X)

/home/soa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/soa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/soa/anaconda3/lib/python3.5/site-packages/numpy/lib/twodim_base.py:233: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m = zeros((N, M), dtype=dtype)
/home/soa/anaconda3/lib/python3.5/site-packages/numpy/lib/twodim_base.py:240: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m[:M-k].flat[i::M+1] = 1
/home/soa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the f

In [3]:
s = system_model.Simulation(M.model,np.zeros(ns))
css = control.ss(ss.A, ss.B, np.eye(ns), np.zeros([ns,ns/2]))
css.B = css.B[:,2:]

[w,v] = np.linalg.eig(ss.A)
Q = np.diag([1,1,100,1,1,1])
Q[:2,:2] = np.asarray([[10,-10],[-10,10]])
Q[3:5,3:5] = np.asarray([[10,-10],[-10,10]]) * 10
[K,S,E] = control.lqr(css, Q, 200*np.eye(1))
K = np.r_[np.zeros([2,ns]),K]
print(K)

P = linear_controller.LinearController( K )
s.control = P
s.x0[0] = -.7
s.x0[1] = .9
s.reset()
print(s.x0)
X = s.simUntil(tMax = 20)

BB.show_state(X)


/home/soa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


[[   0.            0.            0.            0.            0.            0.        ]
 [   0.            0.            0.            0.            0.            0.        ]
 [ 226.10163173  203.11663233   -0.70710678   63.77619959   55.20173406
    -0.95614081]]
[-0.7  0.9  0.   0.   0.   0. ]


In [50]:
[w,v] = np.linalg.eig(ss.A)
w

array([  0.00000000e+00+0.j        ,   0.00000000e+00+0.j        ,
        -4.53749503e+00+0.j        ,   6.52256027e-16+4.78999253j,
         6.52256027e-16-4.78999253j,   4.53749503e+00+0.j        ])

In [53]:

BB.show_state(show_mode(v[:,5]))

/home/soa/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: ComplexWarning: Casting complex values to real discards the imaginary part


In [ ]:
s.control.computeU(s.x0,s.tau)
s.tau
s.reset()
s.sim(10/30.)

P = linear_controller.LinearController( K, np.zeros(4), np.zeros(2))
s.control = P
s.x0 = np.asarray([.05,0,0,0])
s.reset()
BB.show_state(s.simUntil())


In [ ]:
s.X[1,:]
np.linalg.norm(s.X[:,20])
system_model.stateLimits(1e-3,1e2)(s.X[:,20])

if True and not 